## Berkeley Deep RL Course Notes

### Video Lecture 8

---
__[Berkeley DeepRL Course Lecture 8 Video](https://youtu.be/BB-BhTn6DCM)__

--- 
**Policy Gradient Methods**


#### Policy Gradient Methods

In RL the goal is to maximize the sum of the expected rewards over time. 
- Fixed-horizon episodic (fixed time steps)
- Average cost (average reward)
- Infinite-horizon discount (sum all rewards and multiply by discount, futher rewards are less important)
- Variable-length undiscounted (probability of transitioning to a terminal state- episode can end early)
- Infinite-horizon undiscounted (sum is always finite)
 
All yield similiar algorithms.  

*Focus: fixed-horizon episodic, other can be expressed as a limiting form of this one.*
*Goal: Maximizing the expected value of the sum of rewards.*

Can be used with both deterministic and stochastic policy families.

The way policies are parameterized is analogous to classification or regression.
- Discrete action space: network outputs vector of probabilities
- Continuous action space: network outputs mean and diagonal covariance of Gaussian

#### Policy Gradient Methods:

How can we do this?<br>
Collect a bunch of trajectories, try to make the good ones more probable.<br>
Better, try to make the good actions more probable.<br>
Even better, try to push your actions towards better actions (later topic)<br>

Basis of these policy gradient methods is the score function gradient estimator.  Where you want to compute the gradient of some expectation.  <br>

Score function gradient estimator for policies--> what we are doing is taking the good trajectories and treateing them like supervised examples in classification or regression.<br>

To lower variance use temporal structure: <br>
When we get a big reward--> we want to boost the probabilities of the previous actions<br>
- can add a baseline--> only boost probability if better than average rewards<br>
- introduce a discount factor that ignores delayed effects betweek actions and rewards --> acts like an effective time horizon<br>

#### Vanilla Policy Gradient Algorithm 

1. Do a bunch of batches and each batch you collect a set of trajectories by implementing the current policy.
2. At each timestep in the trajectory you compute the return which is the discounted sum of future rewards following that timestep, for the given trajectory.
3. Compute the advantage estimate: return-baseline, how much better was that action than the average action.
4. Refit the baseline function: baseline should equal the discounted return 
5. Update the policy using a policy gradient estimate: compute the gradient estimator, plug into sgd or adam.

#### Implementation with Autodiff
Use neural networks with tensorflow by implementing a loss function.

#### Value Functions
Q-Function (aka state-value function) expected sum of rewards conditioned on the first state and first action.<br>
Advange Function: conditioned on just the first state (Q-function - value function) tells if the action is better or worse than the average action sampled<br>

Value functions can be added to policy gradient formulas.  Can plug in an advantage estimators with the form of return - V(s).<br>

#### Value Functions in the future
The baseline accounts for and removes the effect of past actions, subtracts out the value of the current state that depends on all the past actions.<br>
The value function removes the effect of the future actions. By subtracting out the value function at the current timestep, we get the advantage estimator.

#### Model Predictive Control Connections
MPC look at a finite horizon problem, compute optimal policy at each timestep, take optimal action at first timestep, then replan... next action...<br>
Can compute a finite horizon Q-function and take the max over actions.<br>
The finite horizon works like a discount factor (finite horizon-hard, discount-soft).<br>

#### Paper: Asynchronous methods for deep reinforcement learning (2016)
Finite-horizon method: actor-critic<br>
Used a fixed-horizon advantage estimator.
1.  Agent acts for T timesteps.
2.  At each timestep: compute returns (discounted sum of rewards up to the last time step) and advantages (that return minus the state value function aproximated by a neural network)
3.  Compute loss gradient- update network
4.  Plug gradient estimator into stochastic gradient descent algorithm.

Found that the high varience was not as big a problem as previously thought. 

__[Nice intuitive explanation of policy gradients](http://karpathy.github.io/2016/05/31/rl/)__

### Video Lecture 9

---
__[Berkeley DeepRL Course Lecture 9 Video](https://youtu.be/Wnl-Qh2UHGg)__

--- 
**Q-Function Learning Methods**

Use value based algorithms to solve for policies. <br>

Model free reinforcement methods that center around value functions- mostly value iteration.<br>

Bellman equation is a consistency function.<br>

**Bellman Backup**
Bellman backup operator: take the right side of bellman equation (looking into the future), maps q-functions to q-functions.  This is what you get if you do 2 steps of finite horizon problem<br>

**Q\* (Q-function of the optimal policy)**<br>
pi\* is the optimal policy for the gamma discounted policy<br>
pi\* is the argmax of the Q\*(s,a)<br>
The value function is just expectations over actions of the Q-function.<br>

The bellman backup is a contraction that converges at Q\*<br>

#### Q-Value Iteration
Implementing like this would mean storing a bunch of numbers- but for simplicity.<br>
1. initialize Q
loop<br>
2. Q = TQ  (T is backup operator)
3. converges at Q\*

Could aslo be done with Greedy policy (G)<br>
1. initialize Q
loop<br>
2. pi^(n+1) = GQ^n
3. Q^(n+1) = Q^pi^(n+1)
3. converges at Q\*

**Q-Modified Policy Iteration**<br>
Instead of solving for the exact Q-function of the policy, you apply k- bellman backups applying t^pi<br>

#### Sample Based Estimates

We use the two Bellman backup formulas to estimate/calculate Q^pi and Q\*, then can apply value iteration.<br>
Q-functions allow an unbiased estimator for the right hand side of these equations using a single sample, does not matter what policy was used to select the actions.<br>
This can be expanded to multi-step estimates.  You can estimate a trajectory or trajectory segment.<br>

*Why are we using Q-functions instead of value functions?*<br>
We are making a model free: we can get an unbiased estimate for the right hand side of the Q-functions by taking one state transition.
- the greedy action can be computed without knowing the probability
- can compute an unbiased estimator of backup value without knowing P, using a single transition
- can compute unbiased estimator of backup value using off-policy data

Sampling-based algorithms
1. start with Q-value/policy iteration 
2. replace backup by estimator
3. add some noise to the backup

#### Least Squares Version of Backup
You can rewrite mean as the minimizer of the sum of squared difference.  This can be applied to Q-values to replace the mean.  This is the same, just a different way to express it.

#### Partial backups
There might be too much noise in previous method.<br>
Instead, move a little bit in the direction by taking a gradient step on the squared error (epsilon).<br>
If you take a small multiple of a noisy gradient step-it will improve the result.<br>

Sampling-Based Q-Value Iteration is a model free method that will converge on the optimal solution.<br>
Be careful with parameters- step sizes.<br>

#### Function Approximation/Neural-Fitted Algorithms
The purpose of this is to do value iteration where we don't know what the transition probability is.  First switch to Q-functions, then derived a Q-value iteration, modified it to minimize the squared bellman error.  Now a function approximator can be substituted for Q-value iteration.<br>

We can have a big neural network that approximates the Q-function.  (neural fitted Q-iteration)<br>
- sample a bunch of trajectories using current policy (greedy policy for the current q function plus some exploration noise), then update q-policy.

